# CREDIT RISK ANALYSIS (DATA CLEANING)


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [5]:
customers = pd.read_csv(r"/content/customers_raw.csv")
loans = pd.read_csv(r"/content/loans_raw.csv")
repayments = pd.read_csv(r"/content/repayments_raw.csv")

def audit(df, name):
  print(df.shape())
  print(df.isnull.sum())
  print(df.isnull().mean().round(3))

# *CUSTOMER TABLE*

In [6]:
customers.head()

,customer_id,age,gender,employment_type,annual_income,credit_score,city_tier
0,1,56,Male,Self-Employed,309000.0,718.0,Tier 3
1,2,69,Female,Unemployed,290000.0,312.0,Tier 3
2,3,46,Female,Unemployed,881000.0,848.0,NaN
3,4,32,Male,NaN,44000.0,563.0,Tier 1
4,5,60,Male,Self-Employed,445000.0,559.0,Tier 1


In [8]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50500 entries, 0 to 50499
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      50500 non-null  int64  
 1   age              50500 non-null  int64  
 2   gender           49518 non-null  object 
 3   employment_type  48033 non-null  object 
 4   annual_income    46457 non-null  float64
 5   credit_score     47976 non-null  float64
 6   city_tier        37896 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 2.7+ MB


In [13]:
customers.isnull().sum()

,0
customer_id,0
age,0
gender,0
employment_type,2467
annual_income,4043
credit_score,2524
city_tier,12604


In [11]:
customers['gender'] = customers['gender'].ffill()

In [12]:
customers['gender'].isnull().sum()

np.int64(0)

In [14]:
customers['employment_type'] = customers['employment_type'].fillna(customers['employment_type'].mode()[0])

In [15]:
customers['employment_type'].isnull().sum()

np.int64(0)

In [18]:
customers['annual_income'] = customers['annual_income'].fillna(customers['annual_income'].median())

In [19]:
customers['annual_income'].isnull().sum()

np.int64(0)

In [21]:
customers['credit_score'] = customers['credit_score'].fillna(customers['credit_score'].median())

In [23]:
customers.drop('city_tier', axis=1, inplace=True)

In [24]:
customers.isnull().sum()

,0
customer_id,0
age,0
gender,0
employment_type,0
annual_income,0
credit_score,0


In [25]:
for i in customers.columns:
  print(customers[i].value_counts())

customer_id
24310    2
39948    2
39947    2
11880    2
21173    2
        ..
16736    1
16737    1
16738    1
16739    1
16727    1
Name: count, Length: 50000, dtype: int64
age
30    1055
65    1051
34    1049
40    1036
64    1022
32    1020
43    1017
69    1014
35    1007
55    1007
46    1006
45     993
61     983
27     982
38     979
21     977
33     977
56     976
26     975
41     974
68     974
42     974
62     974
39     972
20     971
19     969
22     968
54     967
59     967
53     966
47     965
36     965
49     964
57     962
58     958
25     955
24     953
31     953
28     948
66     948
52     944
50     940
29     940
44     940
48     931
67     930
23     929
37     927
63     924
18     920
51     911
60     891
Name: count, dtype: int64
gender
Female    24826
Male      24725
Other       949
Name: count, dtype: int64
employment_type
Salaried         30330
Self-Employed    12508
Unemployed        7662
Name: count, dtype: int64
annual_income
601000.0     4093


In [26]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50500 entries, 0 to 50499
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      50500 non-null  int64  
 1   age              50500 non-null  int64  
 2   gender           50500 non-null  object 
 3   employment_type  50500 non-null  object 
 4   annual_income    50500 non-null  float64
 5   credit_score     50500 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.3+ MB


In [29]:
def credit_segment(score):
  if pd.isna(score):
    return 'Unknown'
  elif score < 570:
    return 'Poor'
  elif score < 650:
    return 'Fair'
  elif score < 720:
    return 'Good'
  else:
    return 'Excellent'

customers['credit_segment'] = customers['credit_score'].apply(credit_segment)

In [30]:
def income_segment(income):
  if pd.isna(income):
    return 'Unknown'
  elif income < 400000:
    return 'Low'
  elif income < 800000:
    return 'Medium'
  else:
    return 'High'

customers['income_segment'] = customers['annual_income'].apply(income_segment)

In [34]:
customers.head()

,age,gender,employment_type,annual_income,credit_score,credit_segment,income_segment
customer_id,,,,,,,
1,56,Male,Self-Employed,309000.0,718.0,Good,Low
2,69,Female,Unemployed,290000.0,312.0,Poor,Low
3,46,Female,Unemployed,881000.0,848.0,Excellent,High
4,32,Male,Salaried,44000.0,563.0,Poor,Low
5,60,Male,Self-Employed,445000.0,559.0,Poor,Medium


In [33]:
customers.set_index('customer_id', inplace=True)

In [55]:
customers['income_in_months'] = customers['annual_income'] / 12

# ***Loans Table***

In [35]:
loans.head()

,loan_id,customer_id,loan_amount,interest_rate,loan_term_months,emi,disbursal_date
0,1,43331,50000,12.60,18,3127.78,2025-02-08
1,2,43331,100000,29.68,36,3602.22,2025-07-08
2,3,9202,300000,12.76,6,56380.00,2025-03-30
3,4,7000,500000,26.45,36,17562.50,2024-03-06
4,5,5013,500000,22.07,24,25431.25,2025-12-23


In [36]:
loans.set_index('loan_id', inplace=True)

In [37]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63098 entries, 1 to 63098
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       63098 non-null  int64  
 1   loan_amount       63098 non-null  int64  
 2   interest_rate     63098 non-null  float64
 3   loan_term_months  63098 non-null  int64  
 4   emi               63098 non-null  float64
 5   disbursal_date    63098 non-null  object 
dtypes: float64(2), int64(3), object(1)
memory usage: 3.4+ MB


In [38]:
loans.isnull().sum()

,0
customer_id,0
loan_amount,0
interest_rate,0
loan_term_months,0
emi,0
disbursal_date,0


In [42]:
loans['interest_rate'] = (loans['interest_rate'].astype(str)
.str.replace('%' , '' , regex = False)
.str.replace('Percent' , '' , regex = False))

loans['interest_rate'] = pd.to_numeric(loans['interest_rate'], errors='coerce')

In [50]:
loans = loans[loans['loan_amount'] > 0]

In [52]:
loans = loans[loans['loan_term_months'] > 0]

In [53]:
loans = loans[(loans['interest_rate'] >= 10) & (loans['interest_rate'] <=40)]

In [57]:
expected_emi =  (
loans['loan_amount'] * (1 + loans['interest_rate']/100) ) / loans['loan_term_months']

emi_diff_pct = abs(loans['emi'] - expected_emi) / expected_emi

loans = loans[emi_diff_pct <= 0.20]

# ***Repayment Status***

In [43]:
repayments.head()

,loan_id,months_paid,default_flag,default_month,recovered_amount
0,1,15,0,NaN,46916.70
1,2,29,0,NaN,104464.38
2,3,3,0,NaN,169140.00
3,4,6,0,NaN,105375.00
4,5,13,0,NaN,330606.25


In [44]:
repayments.set_index("loan_id" , inplace = True)

In [45]:
repayments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63098 entries, 1 to 63098
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   months_paid       63098 non-null  int64  
 1   default_flag      63098 non-null  int64  
 2   default_month     11298 non-null  float64
 3   recovered_amount  63098 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 2.4 MB


In [46]:
repayments.isnull().sum()

,0
months_paid,0
default_flag,0
default_month,51800
recovered_amount,0


In [48]:
del repayments['default_month']

In [49]:
repayments.isnull().sum()

,0
months_paid,0
default_flag,0
recovered_amount,0


In [60]:
repayments['recovered_amount'] = repayments['recovered_amount'].clip(upper=loans['loan_amount'])

In [63]:
repayments = repayments.merge(loans[['loan_amount', 'loan_term_months']],
                              left_index=True,
                              right_index=True,
                              how='inner'
                              )

In [64]:
repayments.head()

,months_paid,default_flag,recovered_amount,loan_amount,loan_term_months
loan_id,,,,,
1,15,0,46916.70,50000,18
2,29,0,100000.00,100000,36
3,3,0,169140.00,300000,6
4,6,0,105375.00,500000,36
5,13,0,330606.25,500000,24


# ***Feature Engineering***

In [65]:
repayments['loss_amount'] = np.where(repayments['default_flag'] == 1 ,
                                     repayments['loan_amount'] - repayments['recovered_amount'] , 0)
repayments['interest_earned'] = repayments['recovered_amount']


In [66]:
repayments.head()

,months_paid,default_flag,recovered_amount,loan_amount,loan_term_months,loss_amount,interest_earned
loan_id,,,,,,,
1,15,0,46916.70,50000,18,0.0,46916.70
2,29,0,100000.00,100000,36,0.0,100000.00
3,3,0,169140.00,300000,6,0.0,169140.00
4,6,0,105375.00,500000,36,0.0,105375.00
5,13,0,330606.25,500000,24,0.0,330606.25


In [70]:
customers = customers[~customers.index.duplicated(keep='first')]
assert customers.index.is_unique
assert loans.index.is_unique
assert repayments.index.is_unique

In [71]:
customers.to_csv('customers_cl.csv')
loans.to_csv('loans_cl.csv')
repayments.to_csv('repayments_cl.csv')